In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [2]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  regex = re.compile('[^A-Za-zА-Яа-я]')
  cleantext = regex.sub(' ', cleantext)
  return cleantext

In [3]:
def gettext(bad): 
    mid = []
    i = 1
    while bad[i].text.split()[0] != 'Материал' and bad[i].text.split()[1] != 'опубликован':
        mid.append(bad[i].text)
        i += 1
        if i > len(mid):
            break
    good = [j for i in mid for j in i.split()]
    return good

In [4]:
def getli(li):
    mid = []
    

In [5]:
def document(id):
    page = requests.get('https://vc.ru/' + str(id) )
    soup = BeautifulSoup(page.text, 'html5lib')
    info = []
    text =[]
    if soup.find("h1", class_="content-header__title") and 'Вакансия' not in soup.find('title').text:
        #titledone
        info.append(soup.find("h1", class_="content-header__title").text.split())
        #text
        if soup.find_all('p') != None:
            bad = soup.find_all('p')
            text.append(gettext(bad))
        if soup.find_all("li") != None: 
            text.append([var for i in soup.find_all("li") for var in i.text.split()])
        else:
            text.append('NaN')
        info.append(text)
        #yeardone
        info.append(soup.find("time", class_= "time").text.split()[2])
        #sectiondone
        info.append(soup.find("div", class_=
                              "content-header-author__name").text.split())
        #votesdone
        a = soup.find("span", class_="vote__value__v vote__value__v--real").text
        if a[0] == '–':
            b = int(a[1:])
            info.append(-b)
        else:
            info.append(int(a))
        #viewsdone
        info.append(soup.find("span", class_="views__value").text)
        #bookmarksdone
        if soup.find('div',
                        class_="favorite_marker favorite_marker--type-content favorite_marker--non_zero"):
            info.append(int(soup.find('div',
                        class_="favorite_marker favorite_marker--type-content favorite_marker--non_zero").text.split()[0]))
        else:
            info.append(0)
        #commentsdone
        comment1 = soup.find_all("div", class_="comments__item__text")
        comments = []
        for i in range(len(comment1)):
            comments.append(comment1[i].text.split())
        info.append(comments)
    else:
        info.append('NaN')
    print(id)
    return info

In [8]:
from multiprocessing import Pool
with Pool(100) as p:
    a = p.map(document, np.arange(2646,7646))

IndexError: list index out of range

In [7]:
df = pd.DataFrame(a, columns=['title', 'text', 'year', 'section', 'votes', 'views', 'bookmarks', 'comments'])
df.dropna(inplace=True)
df

NameError: name 'a' is not defined

In [ ]:
df.to_csv('vcru.csv')
from google.colab import drive
drive.mount('/content/drive')

from google.colab import files
files.download('vcru.csv')

In [12]:
a = document(6832)
a[1]

6832


[['Пользователи',
  'сервиса',
  'вопросов',
  'и',
  'ответов',
  'Quora',
  'обсудили,',
  'что',
  'делать',
  'молодому',
  'предпринимателю,',
  'если',
  'девушка',
  'ставит',
  'перед',
  'ним',
  'выбор:',
  'либо',
  'она,',
  'либо',
  'стартап.'],
 ['Во-первых,',
  'она',
  'не',
  'понимала,',
  'как',
  'много',
  'для',
  'него',
  'значит',
  'бизнес.',
  'Во-вторых,',
  'он',
  'действительно',
  'уделял',
  'работе',
  'гораздо',
  'больше',
  'времени,',
  'чем',
  'девушке.']]